In [2]:
import pandas as pd
import numpy as np
import plotly_express as px

In [3]:
# Load CSV file
filenames = ['round_4/prices_round_4_day_1', 'round_4/prices_round_4_day_2', 'round_4/prices_round_4_day_3']
data = pd.DataFrame()
for i, file in enumerate(filenames):
    df = pd.read_csv(f'data/{file}.csv', index_col='timestamp', sep=';').drop(columns='day')
    df.index = df.index + 1000000 * i
    data = pd.concat([data, df])

# Get data for each product
product_data = {
    product: data[data['product'] == product].drop(columns='product')
    for product in data['product'].unique()
}
print('Available products:', list(product_data.keys()))

Available products: ['COCONUT', 'COCONUT_COUPON']


In [901]:
ret = product_data["COCONUT"]["mid_price"].pct_change(1)
# u = ret[ret>0].rolling(100).mean()+1
# d = ret[ret<0].rolling(100).mean()+1
u = ret.rolling(2000, min_periods=500).apply(lambda x: x[x>0].median())+1
d = ret.rolling(2000, min_periods=500).apply(lambda x: x[x<0].median())+1
_u = u.reindex(ret.index)
_u.ffill(inplace=True)
_d = d.reindex(ret.index)
_d.ffill(inplace=True)

$u = e^{\sigma\sqrt{\delta T}}$

In [902]:
N = T*10000
deltaT = T/N
sigma = np.log(_u/_d)/(2*np.sqrt(deltaT)) * np.sqrt(252)
r = (1+np.sqrt(_u*_d)-1)**(252)-1

In [903]:
import statistics 
norm = statistics.NormalDist()
S = product_data["COCONUT"]["mid_price"]
# sigma = ret.rolling(1500).std()*np.sqrt(10000*252)
d1 = (np.log(S/K) + (r + (sigma**2)/2)*1)/(sigma*np.sqrt(1))
delta = d1.apply(norm.cdf)

In [904]:
def f(sigma, S, K, r, C): 
    d1 = (np.log(S/K) + (r + (sigma**2)/2)*1)/(sigma)
    d2 = d1 - sigma
    return S*norm.cdf(d1) - K*np.exp(r)*norm.cdf(d2) - C

def f_prime(sigma, S, K, r, C):
    d1 = (np.log(S/K) + (r + (sigma**2)/2)*1)/(sigma)
    return S*norm.pdf(d1)
    
def newton_method_standard(x0, num_iterations, **kwargs):
    x = x0
    table = []
    
    for i in range(num_iterations + 1):
        en = abs(f(x, **kwargs))
        
        table.append([i, x, en])
        
        if i < num_iterations:
            x = x - f(x, **kwargs) / f_prime(x, **kwargs)
    
    return table

In [905]:
from tqdm import tqdm
sigma_implied_2 = pd.Series(index=r.index)
for i in tqdm(range(500, len(r))):
    sigma_implied_2.iloc[i] = newton_method_standard(sigma.iloc[i], 100, S=S.iloc[i], K=10000, r=r.iloc[i], C=product_data["COCONUT_COUPON"]["mid_price"].iloc[i])[-1][1]

  0%|          | 62/29500 [00:00<00:47, 619.90it/s]

 75%|███████▌  | 22199/29500 [00:24<00:10, 714.56it/s]

In [ ]:
sigma_diff = (((sigma_implied_2-sigma)<0)*2-1).replace(0, np.nan)

In [ ]:
ptf = pd.DataFrame(columns=["COCONUT","COCONUT_COUPON"], index=ret.index)
sigma_smooth = np.round(sigma_diff.rolling(100).mean()).replace(0, np.nan).ffill()
ptf["COCONUT_COUPON"] = np.maximum(np.minimum(np.round(600*(0.5/delta) * delta/delta)*(sigma_smooth.fillna(0)), 600), -600)
ptf["COCONUT"] = -np.round( (ptf["COCONUT_COUPON"]*delta) * sigma_smooth.abs()).fillna(0)
ptf = ptf.shift(1).fillna(0)

In [ ]:
mid_prices = pd.concat([product_data["COCONUT"]["mid_price"].rename("COCONUT"), product_data["COCONUT_COUPON"]["mid_price"].rename("COCONUT_COUPON")], axis=1)
ask_prices = pd.concat([product_data["COCONUT"]["ask_price_1"].rename("COCONUT"), product_data["COCONUT_COUPON"]["ask_price_1"].rename("COCONUT_COUPON")], axis=1)
bid_prices = pd.concat([product_data["COCONUT"]["bid_price_1"].rename("COCONUT"), product_data["COCONUT_COUPON"]["bid_price_1"].rename("COCONUT_COUPON")], axis=1)
ptf_value = ptf.multiply(mid_prices, axis=1).sum(axis=1)

In [ ]:
ptf_diff = ptf.diff()
cash = -(ptf_diff[ptf_diff>=0].fillna(0).multiply(ask_prices) + ptf_diff[ptf_diff<0].fillna(0).multiply(bid_prices)).fillna(0).sum(axis=1).cumsum()

In [ ]:
px.line(cash+ptf_value)